In [2]:
import json
import time

import numpy as np
import pandas as pd
import requests

import redis
from redis.commands.search.field import NumericField, TextField, TagField, VectorField
from redis.commands.search.query import Query
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

from sentence_transformers import SentenceTransformer

/home/armando/ambientes_python/ambiente_ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
REDIS_HOST = '127.0.0.1'
REDIS_PORT = 6379
REDIS_PASSWORD = ''

conn = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, encoding='utf-8', decode_responses=True)

# JSON Data

In [4]:
res = conn.json().set("anuncio:1", "$", {'titulo': 'Preciosa casa en venta en Lomas de Chapultepec', 
                                           'tipo': 'casa', 'operacion': 'venta', 
                                           'precio': 50000000, 
                                           'amenidades': ['Vigilancia', 'Jardín', 'Alberca'], 
                                           'descripcion': 'Esta casa de 400 m2 se encuentra en una de las mejores zonas exclusivas de la ciudad, cuenta con alberca y jardines'})

res = conn.json().set("anuncio:2", "$", {'titulo': 'Departamento en renta en Roma Norte a tres calles del Metro Sevilla', 
                                           'tipo': 'departamento', 'operacion': 'sevilla', 
                                           'precio': 15000,
                                           'amenidades': ['Vigilancia', 'Parrilla'], 
                                           'descripcion': 'Bonito departamento ubicado en una de las zonas de moda, ideal para personas que trabajan remoto, cercano a cafeterias y restaurantes'})

res = conn.json().set("anuncio:3", "$", {'titulo': 'Departanto en renta con vista al parque', 
                                           'tipo': 'departamento', 'operacion': 'renta', 
                                           'precio': 18000, 
                                           'amenidades': ['Vigilancia', 'Terraza', 'Roof Garden'], 
                                           'descripcion': 'Este departamento con terraza y roof garden tiene además un parque, cuenta con vigilancia y otras amenidades'})

res = conn.json().set("anuncio:4", "$", {'titulo': 'Casa remodelada en venta en Lomas de Chapultepec', 
                                           'tipo': 'casa', 'operacion': 'venta', 
                                           'precio': 60000000, 
                                           'amenidades': ['Jardín', 'Parrilla'], 
                                           'descripcion': 'Preciosa residencia en zona exclusiva, la casa ha sido recientemente remodelada y cuanta con amplios jardines'})

res = conn.json().set("anuncio:5", "$", {'titulo': 'Casa en renta cerca de escuelas en Coyoacán', 
                                           'tipo': 'casa', 'operacion': 'renta', 
                                           'precio': 30000, 
                                           'amenidades': ['Jardín'], 
                                           'descripcion': 'Casa pequeña de 100 m2 pero perfectamente ubicada cerca del centro de Coyoacán, con muchas escuelas cercanas, también cuenta con un pequeño jardín'})

res = conn.json().set("anuncio:6", "$", {'titulo': 'Departamento en renta cerca del centro de Coyoacán', 
                                           'tipo': 'departamento', 'operacion': 'renta', 
                                           'precio': 15000, 
                                           'amenidades': ['Vigilancia', 'Roof Garden', 'Parrilla'], 
                                           'descripcion': 'Departamento en Coyoacán, a dos calles de la plaza principal, con parques y museos a la redonda, cuenta con vigilancia, parrilla y roof garden'})

res = conn.json().set("anuncio:7", "$", {'titulo': 'Departamento en venta cerca en Tlatelolco a unos pasos del metro', 
                                           'tipo': 'departamento', 'operacion': 'venta', 
                                           'precio': 1500000, 
                                           'amenidades': None, 
                                           'descripcion': 'Departamento en venta a unos pasos del metro, en una zona tranquila al norte de la ciudad y accesible económicamente'})

res = conn.json().set("anuncio:8", "$", {'titulo': 'Oportunidad de compra de nuevo departamento en exlusiva zona', 
                                           'tipo': 'departamento', 'operacion': 'venta', 
                                           'precio': 7000000, 
                                           'amenidades': ['Vigilancia', 'Acabados de Lujo', 'Terraza'], 
                                           'descripcion': 'Ven y conoce este nuevo desarrollo en una de las zonas de mayor plusvalia y exclusivas, el departamento cuenta con acabados de lujo y terraza'})

res = conn.json().set("anuncio:9", "$", {'titulo': 'Departamento ideal para estudiantes cerca de UNAM', 
                                           'tipo': 'departamento', 'operacion': 'renta', 
                                           'precio': 7000, 
                                           'amenidades': None, 
                                           'descripcion': 'Este departamento con precio accesible es ideal si eres estudiante de la UNAM, a unos minutos de las facultades'})

res = conn.json().set("anuncio:10", "$", {'titulo': 'Casa Antigua en venta, excelente oportunidad', 
                                           'tipo': 'casa', 'operacion': 'venta', 
                                           'precio': 40000000, 
                                           'amenidades': ['Parrilla', 'Jardín'], 
                                           'descripcion': 'Bonita casa antigua pero bien conservada de 150m2, la casa cuenta con jardín y se ubica en una zona tranquila'})

print(res)

True


# Embeddings

In [5]:
embedder = SentenceTransformer('msmarco-distilbert-base-v4')

In [7]:
keys = sorted(conn.keys("anuncio:*"))

descriptions = conn.json().mget(keys, "$.descripcion")
descriptions = [item for sublist in descriptions for item in sublist]

print(descriptions)
print('')

embeddings = embedder.encode(descriptions).astype(np.float32).tolist()
VECTOR_DIMENSION = len(embeddings[0])

print(VECTOR_DIMENSION)

['Esta casa de 400 m2 se encuentra en una de las mejores zonas exclusivas de la ciudad, cuenta con alberca y jardines', 'Bonita casa antigua pero bien conservada de 150m2, la casa cuenta con jardín y se ubica en una zona tranquila', 'Bonito departamento ubicado en una de las zonas de moda, ideal para personas que trabajan remoto, cercano a cafeterias y restaurantes', 'Este departamento con terraza y roof garden tiene además un parque, cuenta con vigilancia y otras amenidades', 'Preciosa residencia en zona exclusiva, la casa ha sido recientemente remodelada y cuanta con amplios jardines', 'Casa pequeña de 100 m2 pero perfectamente ubicada cerca del centro de Coyoacán, con muchas escuelas cercanas, también cuenta con un pequeño jardín', 'Departamento en Coyoacán, a dos calles de la plaza principal, con parques y museos a la redonda, cuenta con vigilancia, parrilla y roof garden', 'Departamento en venta a unos pasos del metro, en una zona tranquila al norte de la ciudad y accesible económ

In [10]:
pipeline = conn.pipeline()

#EXPORTAR EMBEDDINGS
for key, embedding in zip(keys, embeddings):
    pipeline.json().set(key, "$.descripcion_vector", embedding)
    
pipeline.execute()

[True, True, True, True, True, True, True, True, True, True]

# Indice y esquema

In [20]:
#INDEX GENERAL
try:
    conn.ft("idx:anuncio_vector").dropindex()

except: 
    pass

schema = (
    TextField("$.titulo", as_name="titulo"),
    TagField("$.tipo", as_name="tipo"),
    TagField("$.operacion", as_name="operacion"),
    NumericField("$.precio", as_name="precio"),
    TagField("$.amenidades.*", as_name='amenidades'), 
    TextField("$.descripcion", as_name="descripcion"),
    VectorField(
        "$.descripcion_vector",
        "FLAT",
        {
            "TYPE": "FLOAT32",
            "DIM": VECTOR_DIMENSION,
            "DISTANCE_METRIC": "COSINE",
        },
        as_name="vector",
    )
)

index_definition = IndexDefinition(prefix=["anuncio:"], index_type=IndexType.JSON)
index = conn.ft("idx:anuncio_vector")
res = index.create_index(fields=schema, definition=index_definition)

print(res)

OK


In [21]:
info = conn.ft("idx:anuncio_vector").info()
num_docs = info["num_docs"]
indexing_failures = info["hash_indexing_failures"]

print(f"{num_docs} documents indexed with {indexing_failures} failures")

10 documents indexed with 0 failures


# Busquedas

In [14]:
busquedas = ['Inmuebles en zonas exclusivas', 'Inmuebles accesibles o a precios economicos', 'Inmuebles con vigilancia', 
             'Inmuebles con escuelas cercanas', 'Inmuebles con jardines o cerca de parques']

busquedas_codificadas = embedder.encode(busquedas)
print(len(busquedas_codificadas))


5


In [27]:
#KNN
query = (
    Query('(*)=>[KNN 3 @vector $query_vector AS vector_score]')
     .sort_by('vector_score')
     .return_fields('vector_score', 'titulo', 'operacion', 'tipo', 'descripcion')
     .dialect(2)
)

In [28]:
for i in range(0, len(busquedas)):
    print('Search: ' + busquedas[i])
    
    results = conn.ft('idx:anuncio_vector').search(
        query,
        {
          'query_vector': np.array(busquedas_codificadas[i], dtype=np.float32).tobytes()
        }
    ).docs

    print()
    print('Resultados: \n')
    for result in results:
        print(result['vector_score'])
        print(result['operacion'], result['tipo'])
        print(result['descripcion'])
        print()

    print('\n____________________________________________\n')  

Search: Inmuebles en zonas exclusivas

Resultados: 

0.565960526466
venta departamento
Ven y conoce este nuevo desarrollo en una de las zonas de mayor plusvalia y exclusivas, el departamento cuenta con acabados de lujo y terraza

0.588380217552
venta casa
Esta casa de 400 m2 se encuentra en una de las mejores zonas exclusivas de la ciudad, cuenta con alberca y jardines

0.618106365204
venta casa
Preciosa residencia en zona exclusiva, la casa ha sido recientemente remodelada y cuanta con amplios jardines


____________________________________________

Search: Inmuebles accesibles o a precios economicos

Resultados: 

0.555920958519
renta departamento
Este departamento con precio accesible es ideal si eres estudiante de la UNAM, a unos minutos de las facultades

0.579193174839
venta departamento
Departamento en venta a unos pasos del metro, en una zona tranquila al norte de la ciudad y accesible económicamente

0.670033454895
venta departamento
Ven y conoce este nuevo desarrollo en una d

In [50]:
#RADIO
query = (
    Query('(@vector:[VECTOR_RANGE 0.65 $query_vector])=>{$YIELD_DISTANCE_AS: vector_radio}')
     .sort_by('vector_radio')
     .return_fields('vector_radio', 'titulo', 'operacion', 'tipo', 'descripcion')
     .dialect(2)
)

In [51]:
for i in range(0, len(busquedas)):
    print('Search: ' + busquedas[i])
    
    results = conn.ft('idx:anuncio_vector').search(
        query,
        {
          'query_vector': np.array(busquedas_codificadas[i], dtype=np.float32).tobytes()
        }
    ).docs

    print()
    print('Resultados: \n')
    for result in results:
        print(result['vector_radio'])
        print(result['operacion'], result['tipo'])
        print(result['descripcion'])
        print()

    print('\n____________________________________________\n') 

Search: Inmuebles en zonas exclusivas

Resultados: 

0.565960526466
venta departamento
Ven y conoce este nuevo desarrollo en una de las zonas de mayor plusvalia y exclusivas, el departamento cuenta con acabados de lujo y terraza

0.588380217552
venta casa
Esta casa de 400 m2 se encuentra en una de las mejores zonas exclusivas de la ciudad, cuenta con alberca y jardines

0.618106365204
venta casa
Preciosa residencia en zona exclusiva, la casa ha sido recientemente remodelada y cuanta con amplios jardines

0.647517859936
venta departamento
Departamento en venta a unos pasos del metro, en una zona tranquila al norte de la ciudad y accesible económicamente


____________________________________________

Search: Inmuebles accesibles o a precios economicos

Resultados: 

0.555920958519
renta departamento
Este departamento con precio accesible es ideal si eres estudiante de la UNAM, a unos minutos de las facultades

0.579193174839
venta departamento
Departamento en venta a unos pasos del metr